## Building A Chatbot
In this video We'll go over an example of how to design and implement an LLM-powered chatbot. This chatbot will be able to have a conversation and remember previous interactions.

Note that this chatbot that we build will only use the language model to have a conversation. There are several other related concepts that you may be looking for:

- Conversational RAG: Enable a chatbot experience over an external source of data
- Agents: Build a chatbot that can take actions

This video tutorial will cover the basics which will be helpful for those two more advanced topics.

In [42]:
import os
from dotenv import load_dotenv
load_dotenv() ## aloading all the environment variable
from IPython.display import display, Markdown

groq_api_key=os.getenv("GROQ_API_KEY")
os.environ['OPENAI_API_KEY']=os.getenv("OPENAI_API_KEY")
## Langsmith Tracking
os.environ["LANGCHAIN_API_KEY"]=os.getenv("LANGCHAIN_API_KEY")
os.environ["LANGCHAIN_TRACING_V2"]="true"
os.environ["LANGCHAIN_PROJECT"]=os.getenv("LANGCHAIN_PROJECT")



In [64]:


from langchain_community.llms import Ollama  # Replace ChatOpenAI with Ollama

# model = Ollama(
#     model="deepseek-r1:8b",  # Replace with your model name (run `ollama list` to check)
#     base_url="http://localhost:11434",   
#      format="json",  # Ensures clean output
#     callbacks=[]  # Disables problematic callbacks

# )
from langchain_groq import ChatGroq
model = ChatGroq(model="gemma2-9b-It",
                groq_api_key=groq_api_key,)
model


ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x10f885710>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x10d359410>, model_name='gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [68]:
from langchain_core.messages import HumanMessage
display(model.invoke([HumanMessage(content="Hi , My name is Pritam and I am a Data Scientist")]))

AIMessage(content="Hello Pritam,\n\nIt's nice to meet you! I'm glad to know you're a Data Scientist. \n\nWhat are you working on these days? Do you have any interesting projects you'd like to tell me about?  \n\nI'm always eager to learn more about the world of data science.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 71, 'prompt_tokens': 22, 'total_tokens': 93, 'completion_time': 0.129090909, 'prompt_time': 0.002099231, 'queue_time': 0.24302828799999998, 'total_time': 0.13119014}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--618166f4-2087-4f23-811c-f9555a01e14c-0', usage_metadata={'input_tokens': 22, 'output_tokens': 71, 'total_tokens': 93})

In [70]:
from langchain_core.messages import AIMessage
display(model.invoke(
    [
        HumanMessage(content="Hi , My name is Pritam and I am a Data Scientist"),
        AIMessage(content="Hello Pritam! It's nice to meet you. \n\nAs a Data Scientist, what kind of projects are you working on these days? \n\nI'm always eager to learn more about the exciting work being done in the field of AI.\n"),
        HumanMessage(content="Hey What's my name and what do I do?")
    ]
))

AIMessage(content="You are Pritam, and you are a Data Scientist!  \n\nIs there anything else you'd like me to know about you or your work? 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 36, 'prompt_tokens': 97, 'total_tokens': 133, 'completion_time': 0.065454545, 'prompt_time': 0.005269093, 'queue_time': 0.250203086, 'total_time': 0.070723638}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c6a9d747-56b7-411c-bf5a-8122b4b1e200-0', usage_metadata={'input_tokens': 97, 'output_tokens': 36, 'total_tokens': 133})

### Message History
We can use a Message History class to wrap our model and make it stateful. This will keep track of inputs and outputs of the model, and store them in some datastore. Future interactions will then load those messages and pass them into the chain as part of the input. Let's see how to use this!

In [71]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id:str)->BaseChatMessageHistory:
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model,get_session_history,
                            )

In [72]:
config={"configurable":{"session_id":"chat1"}}

response=with_message_history.invoke(
    [HumanMessage(content="Hi , My name is Pritam Jena and I am a Data Scientist")],
    config=config
)

In [73]:
store

{'chat1': InMemoryChatMessageHistory(messages=[HumanMessage(content='Hi , My name is Pritam Jena and I am a Data Scientist', additional_kwargs={}, response_metadata={}), AIMessage(content="Hello Pritam Jena!\n\nIt's nice to meet you.  \n\nBeing a Data Scientist is a fascinating field. What kind of work do you specialize in? \n\nI'm always eager to learn more about the exciting things people are doing with data.\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 57, 'prompt_tokens': 23, 'total_tokens': 80, 'completion_time': 0.103636364, 'prompt_time': 0.002189401, 'queue_time': 0.244233159, 'total_time': 0.105825765}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c4b9966a-72e4-47aa-841c-efd60d280a52-0', usage_metadata={'input_tokens': 23, 'output_tokens': 57, 'total_tokens': 80})])}

In [75]:
display(response.content)


"Hello Pritam Jena!\n\nIt's nice to meet you.  \n\nBeing a Data Scientist is a fascinating field. What kind of work do you specialize in? \n\nI'm always eager to learn more about the exciting things people are doing with data.\n"

In [76]:
display(with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
))

AIMessage(content='Your name is Pritam Jena.  😊 \n\nI keep track of our conversation so I can provide you with a more personalized experience. \n\n\n\nIs there anything else I can help you with?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 43, 'prompt_tokens': 94, 'total_tokens': 137, 'completion_time': 0.078181818, 'prompt_time': 0.006670252, 'queue_time': 0.24602940700000003, 'total_time': 0.08485207}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--54b2be2a-7745-4ba9-8763-32d78d584be9-0', usage_metadata={'input_tokens': 94, 'output_tokens': 43, 'total_tokens': 137})

In [77]:
## change the config-->session id
config1={"configurable":{"session_id":"chat2"}}
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response

AIMessage(content="As an AI, I have no memory of past conversations and do not know your name.\n\nIf you'd like to tell me your name, I'd be happy to know! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 42, 'prompt_tokens': 12, 'total_tokens': 54, 'completion_time': 0.076363636, 'prompt_time': 0.002037431, 'queue_time': 0.244845239, 'total_time': 0.078401067}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--c2052837-199c-42e0-a666-1889961d3d6e-0', usage_metadata={'input_tokens': 12, 'output_tokens': 42, 'total_tokens': 54})

In [78]:
response=with_message_history.invoke(
    [HumanMessage(content="Hey My name is John")],
    config=config1
)
response.content

"Hi John, it's nice to meet you! 👋 \n\nWhat can I do for you today?"

In [79]:
response=with_message_history.invoke(
    [HumanMessage(content="Whats my name")],
    config=config1
)
response.content

'Your name is John, remember? 😊  \n\nIs there anything else I can help you with?\n'

### Prompt templates
Prompt Templates help to turn raw user information into a format that the LLM can work with. In this case, the raw user input is just a message, which we are passing to the LLM. Let's now make that a bit more complicated. First, let's add in a system message with some custom instructions (but still taking messages as input). Next, we'll add in more input besides just the messages.

In [80]:
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder
prompt=ChatPromptTemplate.from_messages(
    [
        ("system","You are a helpful assistant.Amnswer all the question to the nest of your ability"),
        MessagesPlaceholder(variable_name="messages")
    ]
)

chain=prompt|model

In [81]:
chain.invoke({"messages":[HumanMessage(content="Hi My name is Pritam Jena and I am a Data Scientist")]})

AIMessage(content="Hello Pritam Jena! It's nice to meet you.  \n\nI'm glad you introduced yourself. \n\nI'm ready to answer your questions to the best of my ability. What can I help you with today? \n\nPerhaps you have a data science problem you'd like to discuss, need help with a concept, or are simply curious about something related to my capabilities?  \n\nLet me know! 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 93, 'prompt_tokens': 39, 'total_tokens': 132, 'completion_time': 0.169090909, 'prompt_time': 0.003653041, 'queue_time': 0.245137108, 'total_time': 0.17274395}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--cd889dc8-e971-45e8-a985-05ad5011e27c-0', usage_metadata={'input_tokens': 39, 'output_tokens': 93, 'total_tokens': 132})

In [82]:
with_message_history=RunnableWithMessageHistory(chain,get_session_history)

In [83]:
config = {"configurable": {"session_id": "chat3"}}
response=with_message_history.invoke(
    [HumanMessage(content="Hi My name is pritam Jena and I am a Data Scientist")],
    config=config
)

response

AIMessage(content="Hello Pritam Jena! It's nice to meet you. \n\nI'm happy to help with any questions you have as a data scientist.  \n\nWhat can I do for you today? Are you working on a specific project, looking for information on a particular topic, or just want to chat about data science in general?  \n\nThe more details you provide, the better I can assist you. 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 89, 'prompt_tokens': 39, 'total_tokens': 128, 'completion_time': 0.161818182, 'prompt_time': 0.00253393, 'queue_time': 0.26921938, 'total_time': 0.164352112}, 'model_name': 'gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'finish_reason': 'stop', 'logprobs': None}, id='run--4cc4f3ca-643b-4b46-a26b-e2a1525925c7-0', usage_metadata={'input_tokens': 39, 'output_tokens': 89, 'total_tokens': 128})

In [84]:
response = with_message_history.invoke(
    [HumanMessage(content="What's my name?")],
    config=config,
)

response.content

'Your name is Pritam Jena.  I remember that from our introduction!  \n\nIs there anything else I can help you with, Pritam?\n'

In [85]:
## Add more complexity

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "You are a helpful assistant. Answer all questions to the best of your ability in {language}.",
        ),
        MessagesPlaceholder(variable_name="messages"),
    ]
)

chain = prompt | model

In [86]:
response=chain.invoke({"messages":[HumanMessage(content="Hi My name is Pritam")],"language":"Hindi"})
response.content

'नमस्ते Pritam! 👋 \n\nमुझे आपकी मदद करने में खुशी हो रही है।  आप मुझसे क्या पूछना चाहते हैं? 😊\n'

Let's now wrap this more complicated chain in a Message History class. This time, because there are multiple keys in the input, we need to specify the correct key to use to save the chat history.

In [87]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages"
)

In [88]:
config = {"configurable": {"session_id": "chat4"}}
repsonse=with_message_history.invoke(
    {'messages': [HumanMessage(content="Hi,I am Pritam")],"language":"Hindi"},
    config=config
)
repsonse.content

'नमस्ते Pritam, आपकी मदद करने में मुझे खुशी हो रही है!  आप क्या जानना चाहेंगे? 😊\n'

In [89]:
response = with_message_history.invoke(
    {"messages": [HumanMessage(content="whats my name?")], "language": "Hindi"},
    config=config,
)

In [90]:
response.content

'आपका नाम Pritam है।  आपने ही मुझे बताया था!  \n'

### Managing the Conversation History
One important concept to understand when building chatbots is how to manage conversation history. If left unmanaged, the list of messages will grow unbounded and potentially overflow the context window of the LLM. Therefore, it is important to add a step that limits the size of the messages you are passing in.
'trim_messages' helper to reduce how many messages we're sending to the model. The trimmer allows us to specify how many tokens we want to keep, along with other parameters like if we want to always keep the system message and whether to allow partial messages

In [91]:
from langchain_core.messages import SystemMessage,trim_messages
trimmer=trim_messages(
    max_tokens=45,
    strategy="last",
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on="human"
)
messages = [
    SystemMessage(content="you're a good assistant"),
    HumanMessage(content="hi! I'm bob"),
    AIMessage(content="hi!"),
    HumanMessage(content="I like vanilla ice cream"),
    AIMessage(content="nice"),
    HumanMessage(content="whats 2 + 2"),
    AIMessage(content="4"),
    HumanMessage(content="thanks"),
    AIMessage(content="no problem!"),
    HumanMessage(content="having fun?"),
    AIMessage(content="yes!"),
]
trimmer.invoke(messages)

/Users/pritamdevadattajena/Desktop/pritam_work/Build/genai_env/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[SystemMessage(content="you're a good assistant", additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like vanilla ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='nice', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='whats 2 + 2', additional_kwargs={}, response_metadata={}),
 AIMessage(content='4', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='thanks', additional_kwargs={}, response_metadata={}),
 AIMessage(content='no problem!', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='having fun?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='yes!', additional_kwargs={}, response_metadata={})]

In [92]:
from operator import itemgetter

from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter("messages")|trimmer)
    | prompt
    | model
    
)

response=chain.invoke(
    {
    "messages":messages + [HumanMessage(content="What ice cream do i like")],
    "language":"English"
    }
)
response.content

"As an AI, I don't have access to your personal information, including your ice cream preferences. \n\nWhat's your favorite flavor?  🍦 😊\n"

In [93]:
response = chain.invoke(
    {
        "messages": messages + [HumanMessage(content="what math problem did i ask")],
        "language": "English",
    }
)
response.content

'You asked me "whats 2 + 2" 😊  \n\n\n\nLet me know if you\'d like to try another one!\n'

In [94]:
## Lets wrap this in the MEssage History
with_message_history = RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key="messages",
)
config={"configurable":{"session_id":"chat5"}}

In [95]:
response = with_message_history.invoke(
    {
        "messages": messages + [HumanMessage(content="whats my name?")],
        "language": "English",
    },
    config=config,
)

response.content

"As an AI, I don't have access to past conversations or personal information about you. So I don't know your name. 😊\n\nWould you like to tell me your name?  \n\n"

In [96]:
response = with_message_history.invoke(
    {
        "messages": [HumanMessage(content="what math problem did i ask?")],
        "language": "English",
    },
    config=config,
)

response.content

"As a large language model, I have no memory of past conversations. If you'd like to ask me a math problem, I'm happy to help! 😊  \n\n"